# Распределение товаров по категориям

## Инициализация

In [2]:
import os
import sys

from django.utils import timezone

sys.path.append('/home/ubuntu/anodos.ru/anodos/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'anodos.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


import re
import catalog.runner
from catalog.models import *

In [5]:
products = Product.objects.all()

In [6]:
len(products)

5666

In [ ]:
for product in products:
    print(product.article, product.)